In [2]:
import pandas as pd

songs = pd.read_csv('../data/df_audio_features_1000.csv')
songs.head()

,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature,id,html
0,All Shook Up,Elvis Presley,0.624,0.468,10,-12.162,1,0.1320,0.881,0.000006,0.144,0.952,74.139,NaN,117080,4,5ueyLj6e6oVaTY0KQ6yLaA,https://open.spotify.com/track/5ueyLj6e6oVaTY0...
1,I've Got You Under My Skin - Remastered 1998,Frank Sinatra,0.585,0.247,1,-12.612,1,0.0400,0.452,0.000009,0.107,0.591,127.150,NaN,223760,4,3aEJMh1cXKEjgh52claxQp,https://open.spotify.com/track/3aEJMh1cXKEjgh5...
2,Smoke Gets In Your Eyes,The Platters,0.290,0.227,3,-13.060,1,0.0311,0.944,0.000079,0.617,0.224,114.278,NaN,157293,4,307XEC1IUwUs9ojlEFwH7f,https://open.spotify.com/track/307XEC1IUwUs9oj...
3,"What'd I Say, Pt. 1 & 2",Ray Charles,0.540,0.681,4,-5.440,1,0.0508,0.808,0.000000,0.162,0.794,88.385,NaN,307053,4,5yQ9iMZXGcr5rlO4hoLsP4,https://open.spotify.com/track/5yQ9iMZXGcr5rlO...
4,Dream A Little Dream Of Me,Ella Fitzgerald,0.455,0.167,0,-13.613,1,0.0739,0.918,0.000000,0.173,0.404,76.118,NaN,185067,4,3vFVS2WYHDG4KkWCNecvpn,https://open.spotify.com/track/3vFVS2WYHDG4KkW...


In [4]:
songs_rdy = (
    songs
# createa column that contians name, song and id
    .assign(
        name_song_id = lambda x: x['name'] + ' - ' + x['artist'] + ' - ' + x['id']
    )
    .set_index('name_song_id')
    # select only numeric columns
    .select_dtypes('number')
    # drop columns key, mode, time_signature, duration_ms, type
    .drop(columns=['key', 'mode', 'time_signature', 'duration_ms', 'type'])
    # select only acousticness, danceability
    .loc[:, ['acousticness', 'danceability']]
)
songs_rdy.head()

,acousticness,danceability
name_song_id,,
All Shook Up - Elvis Presley - 5ueyLj6e6oVaTY0KQ6yLaA,0.881,0.624
I've Got You Under My Skin - Remastered 1998 - Frank Sinatra - 3aEJMh1cXKEjgh52claxQp,0.452,0.585
Smoke Gets In Your Eyes - The Platters - 307XEC1IUwUs9ojlEFwH7f,0.944,0.290
"What'd I Say, Pt. 1 & 2 - Ray Charles - 5yQ9iMZXGcr5rlO4hoLsP4",0.808,0.540
Dream A Little Dream Of Me - Ella Fitzgerald - 3vFVS2WYHDG4KkWCNecvpn,0.918,0.455


In [7]:
from sklearn.cluster import KMeans

# create 4 clusters
kmeans = KMeans(n_clusters=4)

# fit the model
kmeans.fit(songs_rdy)

# create a column with the cluster labels
songs_rdy['cluster'] = kmeans.labels_

songs_rdy.head()

/Users/joanclaverol/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,acousticness,danceability,cluster
name_song_id,,,
All Shook Up - Elvis Presley - 5ueyLj6e6oVaTY0KQ6yLaA,0.881,0.624,3
I've Got You Under My Skin - Remastered 1998 - Frank Sinatra - 3aEJMh1cXKEjgh52claxQp,0.452,0.585,3
Smoke Gets In Your Eyes - The Platters - 307XEC1IUwUs9ojlEFwH7f,0.944,0.290,2
"What'd I Say, Pt. 1 & 2 - Ray Charles - 5yQ9iMZXGcr5rlO4hoLsP4",0.808,0.540,3
Dream A Little Dream Of Me - Ella Fitzgerald - 3vFVS2WYHDG4KkWCNecvpn,0.918,0.455,2


In [16]:
songs_rdy_centroids = pd.DataFrame(kmeans.cluster_centers_, columns=songs_rdy.columns).drop(columns='cluster')
songs_rdy_centroids = songs_rdy_centroids.rename(columns=lambda x: x + '_centroid')
songs_rdy_centroids = songs_rdy_centroids.reset_index().rename(columns={'index': 'cluster'})

# rename {col} to {col}_centroid
songs_rdy_centroids

,cluster,acousticness_centroid,danceability_centroid
0,0,0.079484,0.474363
1,1,0.152570,0.758049
2,2,0.918274,0.266273
3,3,0.710003,0.582652


In [28]:
songs_rdy_with_centroids = (
songs_rdy
.merge(songs_rdy_centroids, on='cluster', how='left')
)
songs_rdy_with_centroids['songs_name'] = songs_rdy.index
songs_rdy_with_centroids = songs_rdy_with_centroids.set_index('songs_name')
songs_rdy_with_centroids.head()

,acousticness,danceability,cluster,acousticness_centroid,danceability_centroid
songs_name,,,,,
All Shook Up - Elvis Presley - 5ueyLj6e6oVaTY0KQ6yLaA,0.881,0.624,3,0.710003,0.582652
I've Got You Under My Skin - Remastered 1998 - Frank Sinatra - 3aEJMh1cXKEjgh52claxQp,0.452,0.585,3,0.710003,0.582652
Smoke Gets In Your Eyes - The Platters - 307XEC1IUwUs9ojlEFwH7f,0.944,0.290,2,0.918274,0.266273
"What'd I Say, Pt. 1 & 2 - Ray Charles - 5yQ9iMZXGcr5rlO4hoLsP4",0.808,0.540,3,0.710003,0.582652
Dream A Little Dream Of Me - Ella Fitzgerald - 3vFVS2WYHDG4KkWCNecvpn,0.918,0.455,2,0.918274,0.266273


In [30]:
(
songs_rdy_with_centroids
    .assign(
        distance = lambda x: (x['acousticness'] - x['acousticness_centroid'])**2 + (x['danceability'] - x['danceability_centroid'])**2
    )
    .groupby('cluster')
        # get top 5 songs with the smallest distance
    .apply(lambda x: x.nsmallest(5, 'distance'))
)

acousticness  \
cluster songs_name                                                         
0       Band On The Run - Remastered 2010 - Wings - 1H4...        0.0832   
        Cool Out - Leroy Hutson - 489alJQIWUauqzKTMuXRjm          0.0708   
        Because the Night - Patti Smith - 0lIoY4ZQsdn5Q...        0.0542   
        Hustlin' - Rick Ross - 3UaqMekEh1kbVu2A6DwiQs             0.0849   
        Heart Attack - Demi Lovato - 1V6gIisPpYqgFeWbML...        0.0738   
1       LA CANCIÓN - J Balvin - 0fea68AdmYNygeTGI4RC18            0.1520   
        Que Va - Alex Sensation - 6J1R5wtPXiHiwiPstOSI56          0.1480   
        Guerrera - DELLAFUENTE - 3Q6zROQvMXRtR9NYfDpNWL           0.1650   
        Mayores - Becky G - 2AY1UAimvTqjJC8vDJsOyy                0.1450   
        Jerry Sprunger (with T-Pain) - Tory Lanez - 6Js...        0.1370   
2       An Ending (Ascent) - Remastered 2005 - Brian En...        0.9320   
        Symphony No. 6 in A Minor, "Tragic": I. Allegro...        0.9250   
        I'll Never Smile Again - Sarah Vaughan - 3Sqv7K...        0.9390   
        Concerto for Orchestra (1994 Digital Remaster):...        0.9440   
        Beethoven: Symphony No. 7 in A Major, Op. 92: I...        0.9270   
3       Just the Way You Are - Billy Joel - 49MHCPzvMLX...        0.7030   
        Hipsippy Blues - Art Blakey & The Jazz Messenge...        0.7130   
        Splanky - Count Basie - 3Y0LpU5kqQ3vSZvwcGjl1M            0.7250   
        Crazy Little Thing Called Love - Remastered 201...        0.7130   
        Come Go With Me - The Del-Vikings - 4IYyGIbMS5Z...        0.7130   

                                                            danceability  \
cluster songs_name                                                         
0       Band On The Run - Remastered 2010 - Wings - 1H4...         0.479   
        Cool Out - Leroy Hutson - 489alJQIWUauqzKTMuXRjm           0.453   
        Because the Night - Patti Smith - 0lIoY4ZQsdn5Q...         0.466   
        Hustlin' - Rick Ross - 3UaqMekEh1kbVu2A6DwiQs              0.502   
        Heart Attack - Demi Lovato - 1V6gIisPpYqgFeWbML...         0.504   
1       LA CANCIÓN - J Balvin - 0fea68AdmYNygeTGI4RC18             0.754   
        Que Va - Alex Sensation - 6J1R5wtPXiHiwiPstOSI56           0.766   
        Guerrera - DELLAFUENTE - 3Q6zROQvMXRtR9NYfDpNWL            0.760   
        Mayores - Becky G - 2AY1UAimvTqjJC8vDJsOyy                 0.745   
        Jerry Sprunger (with T-Pain) - Tory Lanez - 6Js...         0.760   
2       An Ending (Ascent) - Remastered 2005 - Brian En...         0.257   
        Symphony No. 6 in A Minor, "Tragic": I. Allegro...         0.248   
        I'll Never Smile Again - Sarah Vaughan - 3Sqv7K...         0.279   
        Concerto for Orchestra (1994 Digital Remaster):...         0.268   
        Beethoven: Symphony No. 7 in A Major, Op. 92: I...         0.291   
3       Just the Way You Are - Billy Joel - 49MHCPzvMLX...         0.589   
        Hipsippy Blues - Art Blakey & The Jazz Messenge...         0.573   
        Splanky - Count Basie - 3Y0LpU5kqQ3vSZvwcGjl1M             0.583   
        Crazy Little Thing Called Love - Remastered 201...         0.599   
        Come Go With Me - The Del-Vikings - 4IYyGIbMS5Z...         0.566   

                                                            cluster  \
cluster songs_name                                                    
0       Band On The Run - Remastered 2010 - Wings - 1H4...        0   
        Cool Out - Leroy Hutson - 489alJQIWUauqzKTMuXRjm          0   
        Because the Night - Patti Smith - 0lIoY4ZQsdn5Q...        0   
        Hustlin' - Rick Ross - 3UaqMekEh1kbVu2A6DwiQs             0   
        Heart Attack - Demi Lovato - 1V6gIisPpYqgFeWbML...        0   
1       LA CANCIÓN - J Balvin - 0fea68AdmYNygeTGI4RC18            1   
        Que Va - Alex Sensation - 6J1R5wtPXiHiwiPstOSI56          1   
        Guerrera - DELLAFUENTE - 3Q6zROQvMXRtR9NYfDpNWL           1   
    

Core concepts: 

* Distances - Food ratings
* Kmeans to create clusters and how it works
* Evaluation of custers - inertia and silhouette score 